Let's activate our previously created environment and import BattMo into the kernel.

In [10]:
import Pkg
Pkg.activate("../") # Activates the environment in the current folder

using BattMo, GLMakie

  Activating project at `c:\Users\lorenah\Documents\Repositories\BattMo-workshop-2025-06-13`


### Part 1 - Create your own parameter sets

This part of the tutorial walks you through the process of creating and customizing your own parameter sets in BattMo.jl. Parameter sets define the physical and chemical properties of the battery system you're simulating. You can build them from scratch using model templates, modify them, and save them for future use.

##### Part A - Initiate a model setup

First, define the battery model configuration you'd like to use. This will serve as the template for generating your parameter set. BattMo includes several default setups to choose from.

In [11]:
model_settings = load_model_settings(; from_default_set = "P2D")
model_settings["SEIModel"] = "Bolay"

model_setup = LithiumIonBattery(; model_settings)

✔️ Validation of ModelSettings passed: No issues found.
──────────────────────────────────────────────────


LithiumIonBattery("P2D Setup object for a lithium-ion model", {
    "SEIModel" => "Bolay"
    "RampUp" => "Sinusoidal"
    "TransportInSolid" => "FullDiffusion"
    "ModelFramework" => "P2D"
}, true)

##### Part B - load an empty parameter set

Next, create an empty parameter dictionary based on your model setup. This will include all the required keys but without any values filled in.

In [12]:
empty_cell_parameter_set = load_cell_parameters(; from_model_template = model_setup)

{
    "Electrolyte" =>     {
        "TransferenceNumber" => 0.0
        "DiffusionCoefficient" => 0.0
        "IonicConductivity" => 0.0
        "Density" => 0.0
        "ChargeNumber" => 0.0
        "Concentration" => 0.0
    }
    "Cell" =>     {
        "ElectrodeGeometricSurfaceArea" => 0.0
        "Case" => ""
    }
    "PositiveElectrode" =>     {
        "ActiveMaterial" =>         {
            "ActivationEnergyOfDiffusion" => 0.0
            "NumberOfElectronsTransfered" => 0.0
            "StoichiometricCoefficientAtSOC0" => 0.0
            "OpenCircuitPotential" => 0.0
            "ReactionRateConstant" => 0.0
            "MassFraction" => 0.0
            "StoichiometricCoefficientAtSOC100" => 0.0
            "ActivationEnergyOfReaction" => 0.0
            "MaximumConcentration" => 0.0
            "VolumetricSurfaceArea" => 0.0
            "DiffusionCoefficient" => 0.0
            "ParticleRadius" => 0.0
            "Density" => 0.0
            "ElectronicConductivity" => 0

##### Part C - Save the empty set to a JSON file

You can now write this empty set to a JSON file. This file can be edited manually, shared, or used as a base for further customization.

In [13]:
file_path = "my_custom_parameters.json"
write_to_json_file(file_path, empty_cell_parameter_set)

Data successfully written to my_custom_parameters.json


##### Part C - Let BattMo help you filling the empty set

If you're unsure about what a specific parameter means or how it should be formatted, BattMo provides a helpful function to inspect any parameter.

In [14]:
print_parameter_info("OpenCircuitPotential")

ℹ️  Parameter Information
Parameter                     type                                    unit
--------------------------------------------------------------------------------
OpenCircuitPotential          String, Dict{String, Vector}, Real      V



The automatic validation when initiating the Simulation object can help you keep track of your progress on filling in the parameter values and find any values that are unrealistic.

In [15]:
cell_parameters = load_cell_parameters(; from_file_path= "my_custom_parameters.json")


sim = Simulation(model_setup, cell_parameters, load_cycling_protocol(;from_default_set= "CCDischarge"));

🔍 Validation of CellParameters failed with 48 issues:

──────────────────────────────────────────────────
──────────────────────────────────────────────────
Issue 1:
📍 Where:       [Electrolyte][TransferenceNumber]
🔢 Provided:    0.0
🔑 Rule:        minimum = 0.05
🛠  Issue:       Value is below the minimum allowed (0.05)

──────────────────────────────────────────────────
Issue 2:
📍 Where:       [Electrolyte][Density]
🔢 Provided:    0.0
🔑 Rule:        minimum = 100.0
🛠  Issue:       Value is below the minimum allowed (100.0)

──────────────────────────────────────────────────
Issue 3:
📍 Where:       [Electrolyte][ChargeNumber]
🔢 Provided:    0.0
🔑 Rule:        minimum = 1
🛠  Issue:       Value is below the minimum allowed (1)

──────────────────────────────────────────────────
Issue 4:
📍 Where:       [Electrolyte][Concentration]
🔢 Provided:    0.0
🔑 Rule:        minimum = 100
🛠  Issue:       Value is below the minimum allowed (100)

──────────────────────────────────────────────────
Iss